In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from nltk.tokenize import TweetTokenizer
from proce import TextPreprocessor


# Cargar los datos en un DataFrame de pandas
movies_df = pd.read_csv('./data/MovieReviews.csv', sep=',')

# Función para tokenizar los tweets
nltk.download('punkt')
def tokenizer(text):
    tt = TweetTokenizer()
    return tt.tokenize(text)

# Descargando las stopwords
nltk.download('stopwords')
stop_words_complete = list(stopwords.words('spanish')) + list(stopwords.words('english'))


# Crear el pipeline
text_pipeline = Pipeline([
    ('preprocessing', TextPreprocessor() ),
    ('vectorizer', CountVectorizer(tokenizer=tokenizer, stop_words=stop_words_complete, lowercase=True)),
    ('classifier', RandomForestClassifier(random_state=2))
])


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(movies_df['review_es'], movies_df['sentimiento'], test_size=0.2, random_state=42)

# Ajustar el pipeline a los datos de entrenamiento
text_pipeline.fit(X_train, y_train)

# Evaluar el pipeline en los datos de prueba
score = text_pipeline.score(X_test, y_test)
print("Accuracy:", score)


In [142]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from nltk.tokenize import TweetTokenizer
from proce import TextPreprocessor


In [29]:
import enchant
broker = enchant.Broker()
broker.describe()
broker.list_languages()

es_es = enchant.Dict("en_US")
print(es_es.check("hola"))

False


In [143]:

# Cargar los datos en un DataFrame de pandas
movies_df = pd.read_csv('./data/MovieReviews.csv', sep=',')

# Función para tokenizar los tweets
nltk.download('punkt')
def tokenizer(text):
    tt = TweetTokenizer()
    return tt.tokenize(text)

# Descargando las stopwords
nltk.download('stopwords')
stop_words_complete = list(stopwords.words('spanish')) + list(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Esteban\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Esteban\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [144]:

# Crear el pipeline
text_pipeline = Pipeline([
    ('preprocessing', TextPreprocessor() ),
    ('vectorizer', CountVectorizer(tokenizer=tokenizer, stop_words=stop_words_complete, lowercase=True)),
    ('classifier', RandomForestClassifier(random_state=2))
])


Reviews initialized


In [145]:
movies_df.columns

Index(['Unnamed: 0', 'review_es', 'sentimiento'], dtype='object')

In [146]:

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(movies_df['review_es'], movies_df['sentimiento'], test_size=0.2, random_state=42)


In [147]:
y_train

4227    negativo
4676    negativo
800     positivo
3671    negativo
4193    negativo
          ...   
4426    negativo
466     positivo
3092    negativo
3772    negativo
860     positivo
Name: sentimiento, Length: 4000, dtype: object

In [148]:
X_train

4227    No puedo comprender cómo se permitió esta foto...
4676    El escritor y director Robert Downey, Sr., un ...
800     Es la mejor película que he visto en mi vida !...
3671    El resumen de la trama del IMDb hace erróneame...
4193    KK debería atenerse a cantar, toda esta pelícu...
                              ...                        
4426    A veces, cuando la película está pasada por lo...
466     This Roscoe "Fatty" Arbuckle comedy is best re...
3092    Si esto es todo lo que los Showowski tienen qu...
3772    Nunca antes, un elenco tan grande de las perso...
860     Vi esta película con mi familia y fue genial!E...
Name: review_es, Length: 4000, dtype: object

In [149]:
X_train.values

array(['No puedo comprender cómo se permitió esta foto.Es principalmente, si no está completamente, inexacto, así que el personaje principal ni siquiera se asemeja a la historia que lo parece físicamente.Esta película me hizo ir a Internet y pasar por mis libros para asegurarme de que recuerdo la personalidad de Nerón correctamente.Por el amor de Dios, tengo un libro que enumera a Nero como una de las personas más malvadas del mundo y la película me hizo sentir mal por él.Debo decir que el personaje de AgraPina parecía cerca de mi entendimiento de ella, tanto física como psicológicamente.Aunque, todos los actores hicieron un buen trabajo, la película era terrible y estoy seguro de que no veré ningún otro hecho por este triste equipo.',
       'El escritor y director Robert Downey, Sr., un pionero del movimiento de cine metro en la década de 1960, satirizó el mundo de la publicidad de Nueva York Madison Avenue con su comedia de vanguardia "PUTNEY SHAPT". Downey no confina su ridículo a 

In [ ]:
# Ajustar el pipeline a los datos de entrenamiento
text_pipeline.fit(X_train, y_train)

# Evaluar el pipeline en los datos de prueba
score = text_pipeline.score(X_test, y_test)
print("Accuracy:", score)